In [1]:
import json
from tqdm import tqdm

In [12]:
big = "memoryalpha"
small = "stexpanded"
top_k = 10

gold_pairs_file = "./data/gold_pairs/" + big + "-" + small + "_formatted.txt"
matched_communities_file = "./results/matched_communities_leiden/top_10_pairs/" + small + "-" + big + "_top_10_pairs.json"

exact_match_file = "./data/exact_match/" + big + "-" + small + "-formatted.json"

communities_big_file = "./results/communities_leiden/" + big + "/final.txt"
communities_small_file = "./results/communities_leiden/" + small + "/final.txt"

small_communities = []
big_communities = []
gold_pairs = []
matched_communities = []

In [13]:
with (open(communities_big_file) as cbf, open(communities_small_file) as csf,
      open(gold_pairs_file) as gpf, open(matched_communities_file) as mcf):
    
    for line in csf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        small_communities.append(numbers_set)

    for line in cbf:
        numbers_set = {int(num) for num in line.strip().split(" ")}
        big_communities.append(numbers_set)
    
    for line in gpf:
        numbers_list = [int(num) for num in line.strip().split(";")]
        gold_pairs.append(numbers_list)

    matched_communities_load = json.load(mcf)
    matched_communities = [[int(key), [int(values[i][0]) for i in range(0, top_k)]] for key, values in matched_communities_load.items()]

In [14]:
matched_communities

[[0, [0, 367, 27, 2146, 270, 1715, 1794, 153, 1292, 2135]],
 [1, [379, 1037, 333, 829, 827, 69, 2209, 421, 2208, 1036]],
 [2, [856, 857, 855, 365, 858, 2089, 546, 616, 549, 1992]],
 [3, [460, 450, 423, 17, 103, 2009, 2051, 70, 1754, 97]],
 [4, [913, 912, 911, 1167, 445, 1164, 1092, 2056, 910, 97]],
 [5, [3, 1100, 1936, 2279, 2117, 1683, 572, 1323, 2038, 27]],
 [6, [912, 2094, 482, 1665, 910, 2092, 97, 911, 451, 481]],
 [7, [913, 1152, 2089, 911, 482, 1160, 910, 2094, 481, 912]],
 [8, [911, 912, 482, 2245, 2089, 913, 2094, 97, 1163, 214]],
 [9, [912, 911, 2094, 97, 913, 102, 910, 2056, 482, 1665]],
 [10, [648, 97, 1050, 85, 2032, 102, 1154, 912, 123, 934]],
 [11, [97, 521, 506, 469, 123, 2032, 598, 468, 920, 2094]],
 [12, [2101, 1037, 2105, 506, 854, 413, 1036, 2032, 97, 2094]],
 [13, [97, 912, 1661, 648, 1154, 607, 920, 85, 913, 1037]],
 [14, [1843, 69, 1037, 1841, 1845, 469, 333, 413, 913, 1036]],
 [15, [1665, 607, 1037, 287, 294, 1786, 1821, 1664, 2209, 829]],
 [16, [2234, 481, 332, 

In [15]:
with open(exact_match_file) as file:
    exact_match = json.load(file)

gold_not_exact = list()
for p in gold_pairs:
    if [p[0], p[1]] not in exact_match:
        gold_not_exact.append([p[0], p[1]])

In [16]:
found = 0
# all_pairs = len(gold_pairs)
all_pairs = len(gold_not_exact)

# for gold_pair in tqdm(gold_pairs):
for gold_pair in tqdm(gold_not_exact):
    for matched_community in matched_communities:
        community_1 = small_communities[matched_community[0]]
        community_2 = set()
        for i in range(0, top_k):
            community_2.update(big_communities[matched_community[1][i]])
        if gold_pair[1] in community_1 and gold_pair[0] in community_2:
            found += 1
            # print(matched_community[0], matched_community[1], gold_pair)
            break
            
print("Gold pairs in matched communities top " + str(top_k) + ": " + str(found / all_pairs * 100) + "%")

100%|██████████| 148/148 [00:02<00:00, 63.22it/s]

Gold pairs in matched communities top 10: 28.37837837837838%


In [6]:
found_big = 0
found_small = 0
for gold_pair in gold_pairs:
    for big_community in big_communities:
        if gold_pair[0] in big_community:
            found_big += 1
            break
    for small_community in small_communities:
        if gold_pair[1] in small_community:
            found_small += 1
            break

print("Big: " + str(found_big / all_pairs * 100) + "%")
print("Small: " + str(found_small / all_pairs * 100) + "%")

Big: 100.0%
Small: 100.0%
